## Load data from kaggle

In [3]:
!pip install kaggle -q
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c plant-pathology-2020-fgvc7

 98% 765M/779M [00:04<00:00, 157MB/s]
100% 779M/779M [00:04<00:00, 171MB/s]


In [5]:
!unzip -q /content/plant-pathology-2020-fgvc7.zip

In [6]:
!pip -q install tensorflow
!pip install -q keras

# Data preparation

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.resnet50  import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import RandomFlip, RandomRotation

%matplotlib inline

In [8]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
SEED = 2007
BATCH_SIZE = 64
TARGET = ['healthy', 'multiple_diseases',	'rust',	'scab']


np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

In [10]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [11]:
test_df['image_path'] = '/content/images/' + test_df['image_id'] + '.jpg'
train_df['image_path'] = '/content/images/' + train_df['image_id'] + '.jpg'

In [12]:
n_train = int(len(train_df)*0.8)

In [13]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
aug_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.1,
                                        horizontal_flip=True,)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [21]:
train_ds = train_gen.flow_from_dataframe(train_df.iloc[:n_train, :],
                                         x_col='image_path',
                                         y_col=TARGET,
                                         class_mode='raw',
                                         batch_size=BATCH_SIZE,
                                         seed=SEED)
aug_ds = aug_gen.flow_from_dataframe(train_df.iloc[:n_train, :],
                                         x_col='image_path',
                                         y_col=TARGET,
                                         class_mode='raw',
                                         batch_size=BATCH_SIZE,
                                         seed=SEED)
val_ds = test_gen.flow_from_dataframe(train_df.iloc[n_train:, :],
                                      x_col='image_path',
                                      y_col=TARGET,
                                      class_mode='raw',
                                      batch_size=BATCH_SIZE,
                                      seed=SEED,
                                      shuffle=False)
test_ds = test_gen.flow_from_dataframe(test_df, x_col='image_path',
                                        class_mode=None,
                                        batch_size=BATCH_SIZE,
                                        seed=SEED,
                                        shuffle=False)

Found 1456 validated image filenames.
Found 1456 validated image filenames.
Found 365 validated image filenames.
Found 1821 validated image filenames.


# Tuning augmentation

In [15]:
def train_model(train, val=None, lr_tl=0.01, lr_ft=1e-4, drop_rate=0, inner_size=(224,224,3),
                n_epoch=10, fine_tune=True):
    """
    Train parameters of model.

    Args:
        lr_tl: Learning rate for transfer learning.
        model_path: Learning rate for fine tuning.
        drop_rate: Rate of dropout layer.
        inner_size: Size of the inner layer.
        n_epoch: Number of epoches for training.
        augmentation: Augmentation layer.

    Return:
        History of model training.
    """

# Create base model
    base_model = ResNet50(weights='imagenet', input_shape=inner_size, include_top=False)
    base_model.trainable = False
    # Create a new model on top.
    inputs = keras.Input(shape=inner_size)
    x = base_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(drop_rate)(x)
    outputs = keras.layers.Dense(4, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    print('====== Strart transfer-learning ======')
    # Transfer-learning train
    model.compile(
                  optimizer=keras.optimizers.Adam(lr_tl),
                  loss=keras.losses.CategoricalCrossentropy(),
                  metrics=[keras.metrics.CategoricalAccuracy()],
                  )
    tl_history = model.fit(train, epochs=n_epoch, validation_data=val)
    if fine_tune:
        print('====== Strart fine-tuning ======')
        # Fine-tuning train
        base_model.trainable=True

        model.compile(
                      optimizer=keras.optimizers.Adam(lr_ft),
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=[keras.metrics.CategoricalAccuracy()],
                      )
        fn_history = model.fit(train, epochs=n_epoch, validation_data=val)
        return (model, fn_history)
    return (model, tl_history)

**Train model without augmentation.**

In [ ]:
train_inf = train_model(train_ds, val_ds)

94765736/94765736 [==============================] - 5s 0us/step
====== Strart transfer-learning ======
Epoch 1/10
23/23 [==============================] - 88s 3s/step - loss: 1.9386 - categorical_accuracy: 0.5330 - val_loss: 0.9281 - val_categorical_accuracy: 0.7397
Epoch 2/10
23/23 [==============================] - 78s 3s/step - loss: 0.6062 - categorical_accuracy: 0.7988 - val_loss: 0.4882 - val_categorical_accuracy: 0.8301
Epoch 3/10
23/23 [==============================] - 77s 3s/step - loss: 0.4615 - categorical_accuracy: 0.8441 - val_loss: 0.4985 - val_categorical_accuracy: 0.8192
Epoch 4/10
23/23 [==============================] - 76s 3s/step - loss: 0.3803 - categorical_accuracy: 0.8729 - val_loss: 0.4164 - val_categorical_accuracy: 0.8384
Epoch 5/10
23/23 [==============================] - 77s 3s/step - loss: 0.3185 - categorical_accuracy: 0.8887 - val_loss: 0.3782 - val_categorical_accuracy: 0.8603
Epoch 6/10
23/23 [==============================] - 80s 4s/step - loss: 0.33

**Train model with augmentation.**

In [ ]:
aug_train_inf = train_model(aug_ds, val_ds)

====== Strart transfer-learning ======
Epoch 1/10
23/23 [==============================] - 83s 4s/step - loss: 1.9979 - categorical_accuracy: 0.5495 - val_loss: 0.8077 - val_categorical_accuracy: 0.8055
Epoch 2/10
23/23 [==============================] - 74s 3s/step - loss: 0.7931 - categorical_accuracy: 0.7589 - val_loss: 0.4176 - val_categorical_accuracy: 0.8493
Epoch 3/10
23/23 [==============================] - 76s 3s/step - loss: 0.4351 - categorical_accuracy: 0.8530 - val_loss: 0.4311 - val_categorical_accuracy: 0.8384
Epoch 4/10
23/23 [==============================] - 75s 3s/step - loss: 0.3558 - categorical_accuracy: 0.8702 - val_loss: 0.4197 - val_categorical_accuracy: 0.8411
Epoch 5/10
23/23 [==============================] - 73s 3s/step - loss: 0.3389 - categorical_accuracy: 0.8798 - val_loss: 0.4109 - val_categorical_accuracy: 0.8521
Epoch 6/10
23/23 [==============================] - 73s 3s/step - loss: 0.4587 - categorical_accuracy: 0.8434 - val_loss: 0.5471 - val_catego

**Inference: Model traned with data augmentation shows better accuracy.**

# Train best model using full train dataset.

In [17]:
full_train_ds = aug_gen.flow_from_dataframe(train_df,
                                            x_col='image_path',
                                            y_col=TARGET,
                                            class_mode='raw',
                                            batch_size=BATCH_SIZE,
                                            seed=SEED)

Found 1821 validated image filenames.


In [18]:
full_train_inf = train_model(full_train_ds)

94765736/94765736 [==============================] - 1s 0us/step
====== Strart transfer-learning ======
Epoch 1/10
29/29 [==============================] - 89s 3s/step - loss: 1.9173 - categorical_accuracy: 0.5766
Epoch 2/10
29/29 [==============================] - 79s 3s/step - loss: 0.5617 - categorical_accuracy: 0.8155
Epoch 3/10
29/29 [==============================] - 80s 3s/step - loss: 0.4265 - categorical_accuracy: 0.8567
Epoch 4/10
29/29 [==============================] - 79s 3s/step - loss: 0.3571 - categorical_accuracy: 0.8775
Epoch 5/10
29/29 [==============================] - 80s 3s/step - loss: 0.4622 - categorical_accuracy: 0.8336
Epoch 6/10
29/29 [==============================] - 79s 3s/step - loss: 0.3170 - categorical_accuracy: 0.8858
Epoch 7/10
29/29 [==============================] - 79s 3s/step - loss: 0.2756 - categorical_accuracy: 0.9023
Epoch 8/10
29/29 [==============================] - 80s 3s/step - loss: 0.3128 - categorical_accuracy: 0.8962
Epoch 9/10
29/29

In [19]:
full_train_inf[0].save('aug_resnet50.h5')

# Predict

In [22]:
pred = full_train_inf[0].predict(test_ds)

29/29 [==============================] - 63s 2s/step


In [24]:
submission = pd.DataFrame(pred, columns=TARGET, index=test_df['image_id'])
submission.index.name = 'image_id'
submission.to_csv('submission.csv')

**Final score is 0.9475.**